In [1]:
import pandas as pd

In [10]:
dataPath = './DataSets/breast-cancer-wisconsin.data.txt'
columns = ['id', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'output']
features = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9']
df = pd.read_csv(dataPath, names=columns)

In [11]:
df['output'] = df['output'].apply(lambda x : 1 if x == 2 else -1)

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,output
0,1000025,5,1,1,1,2,1,3,1,1,1
1,1002945,5,4,4,5,7,10,3,2,1,1
2,1015425,3,1,1,1,2,2,3,1,1,1
3,1016277,6,8,8,1,3,4,3,7,1,1
4,1017023,4,1,1,3,2,1,3,1,1,1
5,1017122,8,10,10,8,7,10,9,7,1,-1
6,1018099,1,1,1,1,2,10,3,1,1,1
7,1018561,2,1,2,1,2,1,3,1,1,1
8,1033078,2,1,1,1,2,1,1,1,5,1
9,1033078,4,2,1,1,2,1,2,1,1,1


In [ ]:

class NaiveBayes:
    
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    
    def priors(self):
        